In [ ]:
### Building Chatbot With Multiple Tools Using Langgraph
### Create a chatbot with tool capabilities from arxiv, wikipedia search and some functions

from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper

In [ ]:
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=2,doc_content_chars_max=500)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
print(arxiv.name)

In [ ]:
arxiv.invoke("Attention iss all you need")

In [ ]:
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=500)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

In [ ]:
wiki.invoke("What is machine learning")

In [ ]:
from dotenv import load_dotenv
load_dotenv()

import os

os.environ["TAVILY_API_KEY"]=os.getenv("TAVILY_API_KEY")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

In [ ]:
### Tavily Search Tool
from langchain_community.tools.tavily_search import TavilySearchResults

tavily = TavilySearchResults()

In [ ]:
tavily.invoke("Provide me the recent AI news for Feb 3rd 2025")

In [ ]:
### Combine all the tools in the list

tools=[arxiv,wiki,tavily]

In [ ]:
## Initialize my LLM model

from langchain_groq import ChatGroq

llm=ChatGroq(model="qwen/qwen3-32b")

llm_with_tools=llm.bind_tools(tools)

In [ ]:
from pprint import pprint
from langchain_core.messages import AIMessage, HumanMessage
llm_with_tools.invoke([HumanMessage(content=f"What is the recent AI News")])

In [ ]:
llm_with_tools.invoke([HumanMessage(content=f"What is the recent AI News")]).tool_calls

In [ ]:
## State Schema
from typing_extensions import TypedDict
from langchain_core.messages import AnyMessage
from typing import Annotated
from langgraph.graph.message import add_messages
class State(TypedDict):
    messages:Annotated[list[AnyMessage],add_messages]

In [ ]:
### Entire Chatbot With LangGraph
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition

### Node definition
def tool_calling_llm(state:State):
    return {"messages":[llm_with_tools.invoke(state["messages"])]}

# Build graph
builder = StateGraph(State)
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START, "tool_calling_llm")
builder.add_conditional_edges(
    "tool_calling_llm",
    # If the latest message (result) from assistant is a tool call -> tools_condition routes to tools
    # If the latest message (result) from assistant is a not a tool call -> tools_condition routes to END
    tools_condition,
)
builder.add_edge("tools", END)


graph = builder.compile()

# View
#display(Image(graph.get_graph().draw_mermaid_png()))


In [ ]:
messages=graph.invoke({"messages":HumanMessage(content="1706.03762")})
for m in messages['messages']:
    m.pretty_print()

In [21]:
messages=graph.invoke({"messages":HumanMessage(content="Provide me the top 10 recent AI news for Feb 3rd 2026")})
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

Provide me the top 10 recent AI news for Feb 3rd 2026
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (4w4xv5z20)
 Call ID: 4w4xv5z20
  Args:
    query: top 10 recent AI news for Feb 3rd 2026
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "AI Round-Up – February 2026", "url": "https://www.fladgate.com/insights/ai-round-up-february-2026", "content": "Anthropic launches Claude Cowork\n\nAnthropic released Claude Cowork, a new desktop AI agent that accesses shared folders to read, edit or create files, with optional Chrome integration for browser-based tasks. Designed as a collaborative \"coworker,\" it handles document workflows autonomously while maintaining Claude's safety guardrails, targeting productivity in research, coding and content creation. Separa

In [ ]:
messages=graph.invoke({"messages":HumanMessage(content="What is machine learning")})
for m in messages['messages']:
    m.pretty_print()